In [ ]:
import sys
from textx import metamodel_from_file, TextXError

def validate_textx_grammar_and_text(grammar_file, text_file):
    """
    Validates a text file against the given textX grammar.

    param:
        grammar_file: Path to the textX grammar file (.tx).
        text_file: input file to valdate.
    """
    
    #check grammar file if valid (for now this is always valid)
    metamodel = metamodel_from_file(grammar_file)
    print(f"The grammar '{grammar_file}' is valid.")

    try:
        #use the meta-model to see if the input file builds a valid model
        with open(text_file, 'r') as f:
            text_content = f.read()
        model = metamodel.model_from_str(text_content)
        var = model.variables
        print(var)
        print(f"The text file '{text_file}' is valid according to the grammar.")
        requirements = getattr(model, 'requirements', None)
       # for req in requirements:
       #     print((req.req.specification.object))
            #print((req.req.specification.object[0].object.ob))
            #print((req.req.specification.object[1].object.ob))
        
        return model
    except TextXError as e:
        print(f"Error validating text against grammar: {e}")

#base file
grammar_path = r"D:\FOKUS\LESS_2\_LESS_OLD\grammar\less.tx"
#updated grammar file with generate requirements
#text_path= r"D:\FOKUS\LESS\grammar\specs.txt"
text_path= r"D:\FOKUS\LESS_2\_LESS_OLD\src\LESS_Req_Generation\less_testgeneration\Results\berlin_heart.ess"
#text_path= r"D:\FOKUS\LESS_2\_LESS_OLD\EGAS_all\Combined\EGAS_ALL_NLP_LESS_Pair_CORRECT.ess"
validate_textx_grammar_and_text(grammar_path, text_path)

The grammar 'D:\FOKUS\LESS_2\_LESS_OLD\grammar\less.tx' is valid.
[<Variable:plausibility_checks>, <Variable:faults>, <Variable:unintended_acceleration>, <Variable:errors>, <Variable:high_driving_torque>]
The text file 'D:\FOKUS\LESS_2\_LESS_OLD\EGAS_all\Combined\EGAS_ALL_NLP_LESS_Pair_CORRECT.ess' is valid according to the grammar.


<Model:Egas>

#### Literal validation with ground truth

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def normalize_text(text):
    return ''.join(text.split())

def calculate_literal_similarity(file1_path, file2_path):
    text1 = normalize_text(read_file(file1_path))
    text2 = normalize_text(read_file(file2_path))

    matching_chars = sum(1 for a, b in zip(text1, text2) if a == b)
    total_chars = max(len(text1), len(text2))

    similarity = matching_chars / total_chars if total_chars > 0 else 1.0
    return similarity

file1 = "D:\FOKUS\LESS\less_req_mapping_target_TARGET.txt"  # Path to the first text file
file2 = "D:\FOKUS\LESS\less_req_mapping_target_GENERATED_GPT.json"  # Path to the second text file

similarity_score = calculate_literal_similarity(file1, file2)
print(f"Literal similarity between the two files: {similarity_score:.2%}")

#### DYNAMIC VALIDATION OF GENERATED LESS CONTENTS

In [49]:
import json
import os
from pprint import pprint

# Define file paths
#json file that is generated by the llm, this file will be validated
json_file = r"D:\FOKUS\LESS_2\SE_26\Results\requirement_generation\indirect_prompts\3_deepseekV3\prompt14\run2__TAKEN\prompt_14_indirect_deepseek_corrected.json"

#the less grammar file, this will be used for validation
tx_grammar_file = r"D:\FOKUS\LESS\grammar\less.tx"

#dynamic grammar file that will be updated in each iteration. the less requirement in this file will be replaced by the LESS attribute content from the llm generated file
dynamic_grammar = r"D:\FOKUS\LESS_2\_LESS_OLD\src\LESS_Req_Generation\less_testgeneration\grammar\EGas\dynamic_grammar.txt"

In [55]:
import sys
from textx import metamodel_from_file, TextXError, TextXSyntaxError

# Load LLM generated JSON file
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Replace the line in the dynamic.txt file
def replace_line_in_tx(less_line, tx_file_path):
    with open(tx_file_path, 'r') as f:
        tx_content = f.readlines()

    updated_lines = []
    inside_requirements = False
    skip_block = False
    for line in tx_content:
        if "RequirementClassification: SecurityFunctional" in line:
            inside_requirements = True
            updated_lines.append(line)
        elif inside_requirements and "{" in line:
            # replace the less requirement in the dynamic_grammar file with the less line that is read
            updated_lines.append(f"\t\t\t{{\n\t\t\t\t{less_line}\n\t\t\t}}\n")
            skip_block = True  # Skip until the closing '}'
        elif inside_requirements and "}" in line and skip_block:
            skip_block = False
            inside_requirements = False
        elif not skip_block:
            updated_lines.append(line)

    return ''.join(updated_lines)

# Validate the txt file ussing the tx grammar that specified less
def validate_tx(txt_content, tx_grammar_file):
    try:
        mm = metamodel_from_file(tx_grammar_file)
        a = mm.model_from_str(txt_content)
       # pprint(vars(a.requirements[0].req.specification))
       # print("____")
       #pprint(vars(a.requirements[0].req.specification.conditionals))
       # print((a.requirements[0].req.specification.conditionals.condition[0].lit))
       #pprint(vars(a))
        return None  # No errors
    except TextXError as e:
        return str(e)

# Main execution
if __name__ == "__main__":
    validation_errors = []

    # Load JSON data
    json_data = load_json(json_file)


    """
    1. Read individual lines from the LLM generated json
    2. For each line create a specification document (in this case the dynamic_grammar file is updated everytime)
    3. Validate the dynamic_grammar file (against LESS grammar)
    4. When errors occur, log them
    
    This whole process is required because it seems that there is no way to allow textX grammar to move past errors. It will stop at any error.
    
    The program therefore checks which LESS specifications were correclty generated and which were incorrect.
    
    This is then the "Validation" of LESS grammar to judge performance of the LLM
    """
    line_count = 0
    for item in json_data:
        line_count = line_count + 1
        less_line = item['LESS']

        updated_content = replace_line_in_tx(less_line, dynamic_grammar)

        # Validate updated tx file
        #print(updated_content)
        error = validate_tx(updated_content, tx_grammar_file)
        if error:
            validation_errors.append({"Line_number": line_count , "LESS": less_line, "Error": error})


    # Print validation errors
    if validation_errors:
        print("Validation Errors:")
        for error in validation_errors:
            print(f"Line{error['Line_number']}, LESS: {error['LESS']}, Error: {error['Error']}")
    else:
        print("All LESS lines processed and validated successfully.")


Validation Errors:
Line14, LESS: IF unintended acceleration THEN THE Engine_Control_Unit SHALL SWITCH TO safe_state, Error: None:108:19: Unknown object "acceleration" of class "ObjectAttribute"
Line21, LESS: THE Power_Switch SHALL DETECT spoofing OF THE Lamp_switch_on_request BY verifying THE MAC, Error: None:108:44: Unknown object "OF" of class "OBJECT"
Line22, LESS: THE Power_Switch SHALL drop ANY Lamp_switch_on_request THAT ARE detected AS spoofed, Error: None:108:66: Unknown object "RE" of class "OBJECT"


In [ ]:
import torch
print(torch.cuda.is_available())
